When you run a notebook cell that imports from the whendo packages, you need to set the NOTEBOOK FILE ROOT to \$\{workspaceFolder\} so  that imports from top-level packages work. For example, with the property's default value of \$\{fileDirName\}, the imports from dispatcher.etc... will fail since the notebook location is one directory down from top-level.





In [1]:
from datetime import time, datetime, timedelta
from pydantic import BaseModel
import requests
import json
from whendo.core.action import Action
from whendo.core.server import Server
from whendo.core.dispatcher import Dispatcher
from whendo.core.scheduler import Immediately
import whendo.core.schedulers.timed_scheduler as sched_x
import whendo.core.actions.file_action as file_x
import whendo.core.actions.list_action as list_x
import whendo.core.actions.sys_action as sys_x
import whendo.core.actions.dispatch_action as disp_x
from whendo.core.programs.simple_program import PBEProgram
from whendo.core.util import PP, TimeUnit, Dirs, DateTime, Now, DateTime2
from whendo.core.resolver import resolve_action, resolve_server

prefix = "http://192.168.0.45:8000"
prefix = "http://192.168.0.26:8000"
prefix = "http://192.168.0.26:8000"
prefix = "http://127.0.0.1:8000"
def get(path:str):
    cmd = f"{prefix}{path}"
    response = requests.get(cmd)
    return response.json()
def put(path:str, data:BaseModel):
    cmd = f"{prefix}{path}"
    response = requests.put(cmd, data.json())
    PP.pprint(response.json())
def post(path:str, data:BaseModel):
    cmd = f"{prefix}{path}"
    response = requests.post(cmd, data.json())
    PP.pprint(response.json())
def post_dict(path:str, data:dict):
    cmd = f"{prefix}{path}"
    response = requests.post(cmd, json.dumps(data))
    PP.pprint(response.json())
def delete(path:str):
    cmd = f"{prefix}{path}"
    response = requests.delete(cmd)
    return response.json()

In [ ]:
get('/dispatcher/clear')

In [ ]:
get('/dispatcher/load')

In [ ]:
get('/dispatcher/describe_all')

In [2]:
get('/dispatcher/clear')

heart_1 = file_x.FileAppend(file="heartbeat1.txt", payload={'words':'heartbreak hotel'})
post('/actions/heartbeat1', heart_1)
heart_2 = file_x.FileAppend(file="heartbeat2.txt", payload={'words':'nothing but heartaches'})
post('/actions/heartbeat2', heart_2)
heart_3 = file_x.FileAppend(file="heartbeat3.txt", payload={'words':'heart of glass'})
post('/actions/heartbeat3', heart_3)

append_1 = file_x.FileAppend(file="append_1.txt")
post('/actions/append_1', append_1)
append_2 = file_x.FileAppend(file="append_2.txt")
post('/actions/append_2', append_2)
append_3 = file_x.FileAppend(file="append_3.txt", payload={'words':'apply'})
post('/actions/append_3', append_3)

sys_info = sys_x.SysInfo()
post('/actions/sys_info', sys_info)
pause = sys_x.Pause()
post('/actions/pause', pause)
logic_1 = list_x.All(actions=[heart_1, heart_2])
post('/actions/logic1', logic_1)
success = list_x.Success()
post('/actions/success', success)
file_append = file_x.FileAppend(file="boomerang.txt")
post('/actions/file_append', file_append)
mini_info = sys_x.MiniInfo()
post('/actions/mini_info', mini_info)
terminate = list_x.Terminate()
post('/actions/terminate', terminate)
raise_if_0 = list_x.RaiseIfEqual(value=0)
post('/actions/raise_if_0', raise_if_0)

info_append_1 = list_x.All(actions=[sys_info, append_1])
info_append_2 = list_x.All(actions=[mini_info, append_2])
info_append_3 = list_x.All(actions=[sys_info, append_3])
post('/actions/info_append_1', info_append_1)
post('/actions/info_append_2', info_append_2)
post('/actions/info_append_3', info_append_3)

raise_all_1 = list_x.All(actions=[list_x.Arg(data={"result":0}), raise_if_0])
raise_all_2= list_x.All(actions=[list_x.Arg(data={"result":1}), raise_if_0])
post('/actions/raise_all_1', raise_all_1)
post('/actions/raise_all_2', raise_all_2)
raise_uf_1 = list_x.UntilFailure(actions=[list_x.Arg(data={"result":0}), raise_if_0])
raise_uf_2= list_x.UntilFailure(actions=[list_x.Arg(data={"result":1}), raise_if_0])
post('/actions/raise_uf_1', raise_uf_1)
post('/actions/raise_uf_2', raise_uf_2)
raise_us_1 = list_x.UntilSuccess(actions=[list_x.Arg(data={"result":0}), raise_if_0])
raise_us_2 = list_x.UntilSuccess(actions=[list_x.Arg(data={"result":1}), raise_if_0])
post('/actions/raise_us_1', raise_us_1)
post('/actions/raise_us_2', raise_us_2)

execute_action = disp_x.Exec(server_name="home", action_name="file_append")
post('/actions/execute_action', execute_action)

scheduler = sched_x.Randomly(time_unit=TimeUnit.second, low=5, high=10)
post('/schedulers/randomly_soon', scheduler)
scheduler = sched_x.Timely(interval=1)
post('/schedulers/often', scheduler)
morning, evening = time(6,0,0), time(18,0,0)
scheduler = sched_x.Timely(interval=1, start=morning, stop=evening)
post('/schedulers/timely_day', scheduler)
scheduler = sched_x.Timely(interval=1, start=evening, stop=morning)
post('/schedulers/timely_night', scheduler)
scheduler = Immediately()
post('/schedulers/immediately', scheduler)

program = PBEProgram().prologue("heartbeat1").epilogue("heartbeat3").body_element("often", "heartbeat2")
post('/programs/program1', program)
info_append = PBEProgram().prologue("info_append_1").epilogue("info_append_3").body_element("often", "info_append_2")
post('/programs/info_append', info_append)

home = Server(host='127.0.0.1', port = 8000, tags = {'program_name': ['info_append'], 'server_name': ['home']})
post('/servers/home', home)
home2 = Server(host='127.0.0.1', port = 8000, tags = {'program_name': ['info_appoint'], 'server_name': ['home2']})
post('/servers/home2', home2)

if True:
    start = Now.dt()+timedelta(seconds=2)
    stop = start + timedelta(seconds=20)
    datetime2 = DateTime2(dt1=start, dt2=stop)
    post("/programs/info_append/schedule", datetime2)
elif True:
    start = Now.dt()
    stop = start + timedelta(seconds=20)
    datetime2 = DateTime2(dt1=start, dt2=stop)
    post("/programs/program1/schedule", datetime2)
elif True:
    get('/schedulers/often/actions/logic1')
    dt = DateTime(date_time=Now.dt()+timedelta(seconds=10))
    post('/schedulers/often/actions/logic1/expire', dt)
    post('/schedulers/often/actions/heartbeat3/defer', dt)
elif True: # write once to heartbeat1 & heartbeat2
    dt = DateTime(date_time=Now.dt()+timedelta(seconds=10))
    post('/schedulers/immediately/actions/logic1/defer', dt)

'action (heartbeat1) was successfully added'
'action (heartbeat2) was successfully added'
'action (heartbeat3) was successfully added'
'action (append_1) was successfully added'
'action (append_2) was successfully added'
'action (append_3) was successfully added'
'action (sys_info) was successfully added'
'action (pause) was successfully added'
'action (logic1) was successfully added'
'action (success) was successfully added'
'action (file_append) was successfully added'
'action (mini_info) was successfully added'
'action (terminate) was successfully added'
'action (raise_if_0) was successfully added'
'action (info_append_1) was successfully added'
'action (info_append_2) was successfully added'
'action (info_append_3) was successfully added'
'action (raise_all_1) was successfully added'
'action (raise_all_2) was successfully added'
'action (raise_uf_1) was successfully added'
'action (raise_uf_2) was successfully added'
'action (raise_us_1) was successfully added'
'action (raise_us_2)

In [ ]:
get('/jobs/run')

In [ ]:
get('/jobs/stop')

In [ ]:
get('/jobs/count')

In [3]:
import whendo.sdk.client as cl
start_stop = list_x.Arg(data={"start_stop": DateTime2(dt1=Now.dt() + timedelta(seconds=3), dt2= Now.dt() + timedelta(seconds=45))})
schedule_key_tags = disp_x.ExecSuppliedKeyTags(key_tags={"roles":["pivot"]}, action = disp_x.ScheduleProgram(program_name="program1"))
cl.Client(host=home.host, port=home.port).execute_supplied_action(list_x.All(include_processing_info=True, actions=[start_stop, schedule_key_tags]))

{'result': [],
 'extra': {'result': {'result': [],
   'data': {'result': {'mode': 'field',
     'program_name': None,
     'start_stop': {'dt1': '2021-04-24T17:10:18.528881',
      'dt2': '2021-04-24T17:11:00.528953'},
     'schedule_program': 'schedule_program'},
    'info': {'class': 'whendo.core.actions.list_action.Arg',
     'instance': {'arg': 'arg',
      'data': {'mode': 'field',
       'program_name': None,
       'start_stop': {'dt1': '2021-04-24T17:10:18.528881',
        'dt2': '2021-04-24T17:11:00.528953'},
       'schedule_program': 'schedule_program'}}}},
   'info': {'class': 'whendo.core.actions.dispatch_action.ExecSuppliedKeyTags',
    'instance': {'mode': 'field',
     'action': {'mode': 'field',
      'program_name': 'program1',
      'start_stop': None,
      'schedule_program': 'schedule_program'},
     'key_tags': {'roles': ['pivot']},
     'key_tag_mode': 'any',
     'exec_supplied_key_tags': 'exec_supplied_key_tags'}}},
  'processing_count': 2,
  'success_count': 

In [ ]:
from whendo.core.action import Action
from typing import List
class Kompose(Action):
    compose: str = "compose"
    actions: List[Action]

    def description(self):
        return (
            f"Performs function composition on supplied actions in left-to-right order."
        )

    def execute(self, tag: str = None, data: dict = None):
        loop_data = data
        print(loop_data)
        for action in self.actions:
            loop_data = action.execute(tag=tag, data=loop_data)
            print(loop_data)
        return self.action_result(result=self.get_result(loop_data), data=data)

In [ ]:
start_stop = list_x.Arg(data={"start_stop": DateTime2(dt1=Now.dt() + timedelta(seconds=3), dt2= Now.dt() + timedelta(seconds=45))})
schedule_key_tags = disp_x.ExecSuppliedKeyTags(key_tags={"roles":["pivot"]}, action = disp_x.ScheduleProgram(program_name="program1"))
cl.Client(host=home.host, port=home.port).execute_supplied_action(Kompose(include_processing_info=True, actions=[start_stop, schedule_key_tags]))

In [ ]:
import json
Kompose(include_processing_info=True, actions=[start_stop, schedule_key_tags]).dict()

In [ ]:
k=Kompose(include_processing_info=True, actions=[start_stop, schedule_key_tags])
k.json()

In [ ]:
json.dumps(k.json())

In [ ]:
d = {"a":"b", "k":k}
d2 = d.copy()
d2["k"] = k.json()

In [ ]:
json.dumps(d2)

In [ ]:
def jsonify(dictionary:dict):
    for key in dictionary:
        if isinstance(dictionary[key], BaseModel):
            dictionary[key] = dictionary[key].json()


In [ ]:
import whendo.sdk.client as cl
start_stop = list_x.Arg(data={"start_stop": DateTime2(dt1=Now.dt() + timedelta(seconds=3), dt2= Now.dt() + timedelta(seconds=45))})
schedule_key_tags = disp_x.ExecSuppliedKeyTags(key_tags={"roles":["pivot"]}, action = disp_x.ScheduleProgram(program_name="program1"))
schedule_key_tags = disp_x.ExecSuppliedKeyTags(
    key_tags={"roles":["pivot"]},
    action = disp_x.ScheduleProgram(
        program_name="program1",
        start_stop=DateTime2(dt1=Now.dt() + timedelta(seconds=3), dt2= Now.dt() + timedelta(seconds=45))
        )
    )
# list_x.All(include_processing_info=True, actions=[unschedule_key_tags, start_stop, schedule_key_tags])
cl.Client(host=home.host, port=home.port).execute_supplied_action(Kompose(include_processing_info=True, actions=[schedule_key_tags, start_stop]))